# Escherichia Coli exploration

**Author: T. Ruokokoski**

This notebook loads and inspects E. Coli core model.

- Display interactive map of reactions
- Run FBA to determine default maximum biomass production.
- Compare FBA and pFBA
- Display all exhange reactions
- Simulate anaerobic conditions by disabling oxygen uptake.
- Identify essential carbon sources by disabling them one at a time.
- Identify which non-carbon nutrients are essential for growth.
- Test which individual carbon sources (e.g. glucose, lactate, acetate) support growth.
- Inspect details of all reactions
- Inspect specific reactions and metabolites
- Run FVA on exchange reactions

In [1]:
import os
from cobra.io import load_model, read_sbml_model
from cobra import Model
from cobra.flux_analysis import pfba, flux_variability_analysis
import pandas as pd
from IPython.display import display

# Set paths
model_dir = "./models"

pd.set_option('display.max_rows', None)

### Visualizing Core Metabolism

This section loads the *E. coli core* metabolic model and visualizes it using an interactive Escher map. The map shows pathways and allows inspection of reaction fluxes.

In [2]:
from escher import Builder
from cobra.io import to_json

# Load model
model = load_model("textbook")

# Optionally: read from file
#model = read_sbml_model(os.path.join(model_dir, "e_coli_core.xml"))

# Load map as raw JSON string
with open("./models/e_coli_core.Core metabolism.json", "r") as f:
    map_json = f.read()

# Create Escher builder
b = Builder(
    model_json=to_json(model),  # JSON string of the model
    map_json=map_json,          # JSON string of the map
    reaction_scale_width=1.0
)

b


Builder()

## Basic Model Summary

Print statistics: number of reactions, metabolites, and genes for each model.

In [3]:
print(f"Model: {model.id}")
print(f"Reactions: {len(model.reactions)}") # biochemical transformations
print(f"Metabolites: {len(model.metabolites)}") # chemical compounds involved
print(f"Genes: {len(model.genes)}") # genes associated with enzymes that catalyze reactions
display(model.summary())

Model: e_coli_core
Reactions: 95
Metabolites: 72
Genes: 137


Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,4.765,0,0.00%
o2_e,EX_o2_e,21.8,0,0.00%
pi_e,EX_pi_e,3.215,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-22.81,1,100.00%
h2o_e,EX_h2o_e,-29.18,0,0.00%
h_e,EX_h_e,-17.53,0,0.00%


### Run Flux Balance Analysis (FBA)

Perform FBA to compute the maximum biomass production rate under default model constraints. This simulates optimal growth conditions and reveals the most active reactions in the metabolic network.

In [4]:
# Run Flux Balance Analysis (FBA)
default_solution = model.optimize()

# Default objective function is biomass production reaction
print(f"\nMax biomass growth rate: {default_solution.objective_value:.4f} mmol/gDW/hour")

# Get top 10 flux-carrying reactions
top_fluxes = default_solution.fluxes.sort_values(ascending=False).head(10)

top_reactions = []

for rxn_id, flux in top_fluxes.items():
    rxn = model.reactions.get_by_id(rxn_id)
    top_reactions.append({
        "Reaction ID": rxn.id,
        "Name": rxn.name,
        "Equation": rxn.reaction,
        "Flux": round(flux, 4)
    })

df_top = pd.DataFrame(top_reactions)
print("\nTop flux-carrying reactions:")
display(df_top)



Max biomass growth rate: 0.8739 mmol/gDW/hour

Top flux-carrying reactions:


,Reaction ID,Name,Equation,Flux
0,ATPS4r,ATP synthase (four protons for one ATP),adp_c + 4.0 h_e + pi_c <=> atp_c + h2o_c + 3.0...,45.5140
1,CYTBD,cytochrome oxidase bd (ubiquinol-8: 2 protons),2.0 h_c + 0.5 o2_c + q8h2_c --> h2o_c + 2.0 h_...,43.5990
2,NADH16,NADH dehydrogenase (ubiquinone-8 & 3 protons),4.0 h_c + nadh_c + q8_c --> 3.0 h_e + nad_c + ...,38.5346
3,EX_h2o_e,H2O exchange,h2o_e <=>,29.1758
4,EX_co2_e,CO2 exchange,co2_e <=>,22.8098
5,O2t,R o2 - transport-diffusion,o2_e <=> o2_c,21.7995
6,EX_h_e,H+ exchange,h_e <=>,17.5309
7,GAPD,glyceraldehyde-3-phosphate dehydrogenase,g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c,16.0235
8,ENO,enolase,2pg_c <=> h2o_c + pep_c,14.7161
9,GLCpts,D-glucose transport via PEP:Pyr PTS,glc__D_e + pep_c --> g6p_c + pyr_c,10.0000


### Parsimonius Flux Balance Analysis (pFBA)

pFBA finds a flux distribution which gives the optimal growth rate, but minimizes the total sum of flux. Both pFBA and FBA should return identical results within solver tolerances for the objective being optimized.

In [5]:
pfba_solution = pfba(model)

diff = abs(default_solution.fluxes["Biomass_Ecoli_core"] - pfba_solution.fluxes["Biomass_Ecoli_core"])
print(f"Difference in between standard FBA and pFBA: {diff}")

Difference in between standard FBA and pFBA: 3.6637359812630166e-15


### Exchange Reactions Overview

List and inspect all exchange reactions, which represent metabolite uptake and secretion between the cell and its environment. Useful for understanding how the model interfaces with external nutrients and byproducts.

In [6]:
print(f"Number of exchange reactions: {len(model.exchanges)}")
exchange_data = []

for rxn in model.reactions:
    if rxn.id.startswith('EX'):
        exchange_data.append({
            "Reaction ID": rxn.id,
            "Name": rxn.name,
            "Bounds": rxn.bounds,
            "Equation": rxn.reaction,
            "Flux": round(default_solution.fluxes[rxn.id], 4)
        })

df = pd.DataFrame(exchange_data)
display(df)


Number of exchange reactions: 20


,Reaction ID,Name,Bounds,Equation,Flux
0,EX_ac_e,Acetate exchange,"(0.0, 1000.0)",ac_e -->,0.0000
1,EX_acald_e,Acetaldehyde exchange,"(0.0, 1000.0)",acald_e -->,0.0000
2,EX_akg_e,2-Oxoglutarate exchange,"(0.0, 1000.0)",akg_e -->,0.0000
3,EX_co2_e,CO2 exchange,"(-1000.0, 1000.0)",co2_e <=>,22.8098
4,EX_etoh_e,Ethanol exchange,"(0.0, 1000.0)",etoh_e -->,0.0000
5,EX_for_e,Formate exchange,"(0.0, 1000.0)",for_e -->,0.0000
6,EX_fru_e,D-Fructose exchange,"(0.0, 1000.0)",fru_e -->,0.0000
7,EX_fum_e,Fumarate exchange,"(0.0, 1000.0)",fum_e -->,0.0000
8,EX_glc__D_e,D-Glucose exchange,"(-10.0, 1000.0)",glc__D_e <=>,-10.0000
9,EX_gln__L_e,L-Glutamine exchange,"(0.0, 1000.0)",gln__L_e -->,0.0000


### Anaerobic Growth Test

Simulate growth under anaerobic conditions by disabling oxygen uptake and re-running FBA. This tests whether the model can produce biomass without oxygen and reveals how flux distribution changes in its absence.

In [7]:
# Disable oxygen uptake
oxygen = model.reactions.get_by_id('EX_o2_e')
oxygen_lb_backup = oxygen.lower_bound
oxygen.lower_bound = 0.0

# Re-run FBA with no oxygen
anaerobic_solution = model.optimize()

print("\n=== Anaerobic growth test (oxygen uptake disabled) ===")
if anaerobic_solution.status == 'optimal':
    print(f"Biomass without oxygen: {anaerobic_solution.objective_value:.4f}")
    
    # Get top 10 flux-carrying reactions under anaerobic conditions
    top_fluxes_anaerobic = anaerobic_solution.fluxes.sort_values(ascending=False).head(10)

    anaerobic_reactions = []

    for rxn_id, flux in top_fluxes_anaerobic.items():
        rxn = model.reactions.get_by_id(rxn_id)
        anaerobic_reactions.append({
            "Reaction ID": rxn.id,
            "Name": rxn.name,
            "Equation": rxn.reaction,
            "Flux": round(flux, 4)
        })

    df_anaerobic = pd.DataFrame(anaerobic_reactions)
    print("\nTop flux-carrying reactions under anaerobic conditions:")
    display(df_anaerobic)

else:
    print("Optimization failed: no feasible solution without oxygen.")

# Restore original oxygen bound
oxygen.lower_bound = oxygen_lb_backup


=== Anaerobic growth test (oxygen uptake disabled) ===
Biomass without oxygen: 0.2117

Top flux-carrying reactions under anaerobic conditions:


,Reaction ID,Name,Equation,Flux
0,EX_h_e,H+ exchange,h_e <=>,30.5542
1,GAPD,glyceraldehyde-3-phosphate dehydrogenase,g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c,19.4373
2,ENO,enolase,2pg_c <=> h2o_c + pep_c,19.1207
3,FORti,formate transport via diffusion,for_c --> for_e,17.8047
4,PFL,pyruvate formate lyase,coa_c + pyr_c --> accoa_c + for_c,17.8047
5,EX_for_e,Formate exchange,for_e -->,17.8047
6,GLCpts,D-glucose transport via PEP:Pyr PTS,glc__D_e + pep_c --> g6p_c + pyr_c,10.0000
7,PGI,glucose-6-phosphate isomerase,g6p_c <=> f6p_c,9.9566
8,FBA,fructose-bisphosphate aldolase,fdp_c <=> dhap_c + g3p_c,9.7895
9,PFK,phosphofructokinase,atp_c + f6p_c --> adp_c + fdp_c + h_c,9.7895


### Identify Essential Carbon Sources

Test which carbon sources are strictly required for growth by individually disabling their uptake and re-running FBA. Sources whose removal eliminates or severely reduces biomass production are marked as essential.

In [8]:
# Define carbon sources to test
carbon_sources = [
    'glc__D_e',  # Glucose
    'fru_e',     # Fructose
    'lac__D_e',  # D-Lactate
    'pyr_e',     # Pyruvate (intermediate, not a sugar but a carbon source)
    'ac_e',      # Acetate
    'akg_e',     # Alpha-ketoglutarate
    'succ_e',    # Succinate
    'fum_e',     # Fumarate
    'mal__L_e'   # L-Malate
]
carbon_exchanges = [f'EX_{met}' for met in carbon_sources]

# Enable all carbon source uptakes first
for ex in carbon_exchanges:
    if ex in model.reactions:
        model.reactions.get_by_id(ex).lower_bound = -10

# Test if any single carbon source is truly essential
non_redundant = []
threshold = 1e-9
for ex in carbon_exchanges:
    rxn = model.reactions.get_by_id(ex)
    original_lb = rxn.lower_bound
    rxn.lower_bound = 0

    sol = model.optimize()
    if sol.status != 'optimal' or sol.objective_value < threshold:
        non_redundant.append(ex)

    rxn.lower_bound = original_lb  # restore

print("\nCarbon sources that are truly essential:")
print(non_redundant)


Carbon sources that are truly essential:
[]


### Identify Essential Non-Carbon Nutrients

After excluding carbon sources, test which other exchange reactions (e.g., for nitrogen, phosphorus, ions) are essential for biomass production. Each is disabled one at a time to assess its impact on growth.


In [9]:
# Remove carbon sources
uptake_exchanges = [
    rxn for rxn in model.exchanges
    if rxn.lower_bound < 0 and rxn.id not in carbon_exchanges
]
print("\nDefault exchange reactions that allow uptake (excluding carbon sources):")
print([rxn.id for rxn in uptake_exchanges])

# Find which of other exhange reactions are essential
essential = []
for rxn in uptake_exchanges:
    # save original lower bound
    lb_orig = rxn.lower_bound

    # disable uptake
    rxn.lower_bound = 0.0

    sol = model.optimize()
    failed = (sol.status != 'optimal')
    very_low = (sol.objective_value is not None and sol.objective_value < threshold)

    if failed or very_low:
        essential.append(rxn.id)

    # restore original lower bound
    rxn.lower_bound = lb_orig

print("\nMinimal essential composition (excluding carbon source):")
for rxn_id in essential:
    rxn = model.reactions.get_by_id(rxn_id)
    print(f"  {rxn.id}: {rxn.name}")
print(f"\nTotal essential uptake reactions: {len(essential)}")


Default exchange reactions that allow uptake (excluding carbon sources):
['EX_co2_e', 'EX_h_e', 'EX_h2o_e', 'EX_nh4_e', 'EX_o2_e', 'EX_pi_e']

Minimal essential composition (excluding carbon source):
  EX_nh4_e: Ammonia exchange
  EX_pi_e: Phosphate exchange

Total essential uptake reactions: 2


### Growth on Different Sugars

Each carbon source is tested individually by enabling only one sugar uptake at a time. Flux Balance Analysis is used to determine whether the sugar supports biomass production and to compare growth rates across different carbon sources.

In [10]:
print("\n=== Testing growth on different sugars ===")

results = []
for sugar_ex in carbon_exchanges:
    if sugar_ex not in model.reactions:
        print(f"{sugar_ex} not in model, skipping...")
        continue

    for ex in carbon_exchanges:
        if ex in model.reactions:
            model.reactions.get_by_id(ex).lower_bound = 0

    model.reactions.get_by_id(sugar_ex).lower_bound = -10

    sol = model.optimize()

    results.append({
        "Carbon source": sugar_ex,
        "Uptake rate": round(sol.fluxes[sugar_ex], 4),
        "Biomass Flux": round(sol.fluxes['Biomass_Ecoli_core'], 4)
    })

df_sugars = pd.DataFrame(results)
display(df_sugars)


=== Testing growth on different sugars ===


,Carbon source,Uptake rate,Biomass Flux
0,EX_glc__D_e,-10.0,0.8739
1,EX_fru_e,-10.0,0.8739
2,EX_lac__D_e,-10.0,0.3503
3,EX_pyr_e,-10.0,0.2912
4,EX_ac_e,-10.0,0.1733
5,EX_akg_e,-10.0,0.5288
6,EX_succ_e,-10.0,0.3976
7,EX_fum_e,-10.0,0.3707
8,EX_mal__L_e,-10.0,0.3707


### Full Reaction Overview

All reactions in the model are collected into a DataFrame with basic metadata: reaction ID, name, equation, flux bounds, and associated genes. This provides a structured overview for further inspection or export.


In [11]:
reaction_data = []

for rxn in model.reactions:
    reaction_data.append({
        "Reaction ID": rxn.id,
        "Name": rxn.name,
        "Equation": rxn.reaction,
        "Bounds (mmol/gDW/h)": rxn.bounds,
        #"Subsystem": getattr(rxn, "subsystem", "N/A"),
        "Gene Associations": ", ".join(g.id for g in rxn.genes) if rxn.genes else "None"
    })

df_reactions = pd.DataFrame(reaction_data)
display(df_reactions)


,Reaction ID,Name,Equation,Bounds (mmol/gDW/h),Gene Associations
0,ACALD,acetaldehyde dehydrogenase (acetylating),acald_c + coa_c + nad_c <=> accoa_c + h_c + na...,"(-1000.0, 1000.0)","b1241, b0351"
1,ACALDt,R acetaldehyde reversible - transport,acald_e <=> acald_c,"(-1000.0, 1000.0)",s0001
2,ACKr,acetate kinase,ac_c + atp_c <=> actp_c + adp_c,"(-1000.0, 1000.0)","b3115, b1849, b2296"
3,ACONTa,"aconitase (half-reaction A, Citrate hydro-lyase)",cit_c <=> acon_C_c + h2o_c,"(-1000.0, 1000.0)","b0118, b1276"
4,ACONTb,"aconitase (half-reaction B, Isocitrate hydro-l...",acon_C_c + h2o_c <=> icit_c,"(-1000.0, 1000.0)","b0118, b1276"
5,ACt2r,R acetate reversible transport via proton - sy...,ac_e + h_e <=> ac_c + h_c,"(-1000.0, 1000.0)",None
6,ADK1,adenylate kinase,amp_c + atp_c <=> 2.0 adp_c,"(-1000.0, 1000.0)",b0474
7,AKGDH,2-Oxogluterate dehydrogenase,akg_c + coa_c + nad_c --> co2_c + nadh_c + suc...,"(0.0, 1000.0)","b0727, b0726, b0116"
8,AKGt2r,R 2 oxoglutarate reversible transport via - sy...,akg_e + h_e <=> akg_c + h_c,"(-1000.0, 1000.0)",b2587
9,ALCD2x,alcohol dehydrogenase (ethanol),etoh_c + nad_c <=> acald_c + h_c + nadh_c,"(-1000.0, 1000.0)","b1478, b1241, b0356"


### Inspect a Specific Reaction

Access a single reaction from the model by its index or ID.

In [12]:
model.reactions[29]

#model.reactions.get_by_id("EX_glu__L_e")

Reaction identifier,EX_glu__L_e
Name,L-Glutamate exchange
Memory address,0x787855203440
Stoichiometry,glu__L_e --> L-Glutamate -->
GPR,
Lower bound,0.0
Upper bound,1000.0


### Inspect a Specific Metabolite

Retrieve a metabolite from the model using its ID (`"o2_e"` for extracellular oxygen) or index.

In [13]:
model.metabolites.get_by_id("o2_e")

#model.metabolites[3]

Metabolite identifier,o2_e
Name,O2
Memory address,0x7878553ba7b0
Formula,O2
Compartment,e
In 2 reaction(s),"EX_o2_e, O2t"


### Running Flux Variablity Analysis (FVA)

Runs FVA to determine the minimum and maximum flux values that each reaction can carry while maintaining optimal growth. Useful for assessing metabolic flexibility.

In [14]:
exchange_rxns = [rxn for rxn in model.exchanges]

# Run FVA on exchange reactions
flux_variability_analysis(model, exchange_rxns)

,minimum,maximum
EX_ac_e,0.000000,3.747457e-15
EX_acald_e,0.000000,-4.039775e-15
EX_akg_e,0.000000,1.481553e-15
EX_co2_e,24.222952,2.422295e+01
EX_etoh_e,0.000000,2.123559e-15
EX_for_e,0.000000,8.660502e-15
EX_fru_e,0.000000,0.000000e+00
EX_fum_e,0.000000,0.000000e+00
EX_glc__D_e,0.000000,0.000000e+00
EX_gln__L_e,0.000000,0.000000e+00
